In [6]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit #time to process
import requests
import time
from scipy.stats import linregress
from datetime import datetime

from config import weather_api_key
from citipy import citipy # Use the citipy module to determine city based on latitude and longitude.

# Generate Random Lat Long

In [7]:
# Create a set of random latitude and longitude combinations.
# zip creates pairs in the order listed
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [8]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [9]:
len(coordinates)

2000

# Generate Random World Cities
##### Example for iterating through lat long  and finding city

In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

750

# Using API
### Open Weather map API
### import requests 
##### https://requests.kennethreitz.org/en/master/

### Building API URL

In [11]:
# api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}
# api.openweathermap.org/data/2.5/weather?q={city name},{state code},{country code}&appid={API key}
# api.openweathermap.org/data/2.5/weather?q={city name},{state code}&appid={API key}


In [12]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=72c6ac49835b8273bfc5a12e4f65f701


In [13]:
#test if url is working
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url)
city_weather

<Response [429]>

In [14]:
# view data in response to find the keys you need to pull for the dataframe
city_weather.text

'{"cod":429, "message": "Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription http://openweathermap.org/price"}'

In [15]:
city_weather.json()

{'cod': 429,
 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription http://openweathermap.org/price'}

In [16]:
temp_data=city_weather.json()

In [17]:
temp_data['weather'][0]['description']


KeyError: 'weather'

### Initialize an Empty List and Counters

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


In [ ]:
city_data

# Handle API Request Errors with try-except Blocks
### Exception handling

In [ ]:
# Loop through all the cities in the list.
## change to code to run
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50): #is remainder of 50 >= 0... basically checks if there is a remainder. if so contiue counting until there isn't
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+") # replace all spaces in city with "+"

    # Log the URL, record, and set numbers and the city.
    # print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    ###################################################################
    #   Start your exception here...when it adds to the iterator      #
    #   to go to the next record                                      #
    ###################################################################
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        #pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_dirty_df = pd.DataFrame(city_data)
city_data_dirty_df

In [ ]:
city_data_dirty_df.isnull().sum()

In [ ]:
city_data_dirty_df.dropna()

In [ ]:
#convert dirty dataframe to a new cleaned dataframe without nulls
city_data_df = pd.DataFrame(city_data_dirty_df.dropna())

## check if all nulls were completely removed

In [ ]:
city_data_df.isnull().sum()

In [ ]:
city_data_df.notnull().sum()

In [ ]:
city_data_df.count()

### Reorder columns

In [ ]:
### Change order of columns if neccesary
#new_column_order= ["City","Country",'Lat','Lng','Max Temp', 'Humidity','Cloudiness', 'Wind Speed','Current Description']
#city_data_df = city_data_df[new_column_order]

In [ ]:
city_data_df.head(1)

# Convert Dataframe to CSV

In [ ]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")